In [ ]:
import pdfplumber
import os
import csv
from tqdm import tqdm
# PDF 文件夹路径
pdf_folder = './data/A_document/'
# 输出 CSV 文件路径
csv_file_path = './data/A_document.csv'

# 需要删除的文本
remove_text = '本文档为2024CCFBDCI比赛用语料的一部分。部分文档使用大语言模型改写生成，内容可能与现实情况\n不符，可能不具备现实意义，仅允许在本次比赛中使用。\n'

# 初始化 CSV 文件
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    # 写入 CSV 文件的表头
    writer.writerow(['filename', 'content'])
    
    # 遍历文件夹中的所有 PDF 文件
    for filename in tqdm(os.listdir(pdf_folder)):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(pdf_folder, filename)
            # 提取 PDF 文件的内容
            with pdfplumber.open(pdf_path) as pdf:
                content = ""
                for page in pdf.pages:
                    content += page.extract_text() or ""  # 提取每一页的文本并追加
            
            # 删除指定的文本
            content = content.replace(remove_text, "")
            
            # 将文件名和处理后的内容写入 CSV
            writer.writerow([filename, content])

print(f'所有 PDF 文件的内容已提取并保存到 {csv_file_path}')


In [65]:
import pandas as pd
import numpy as np

content_df = pd.read_csv('/data/gyx/2024BDCI/基于运营商文本数据的知识库检索/A_document.csv')
question_df = pd.read_csv('/data/gyx/2024BDCI/基于运营商文本数据的知识库检索/A_question.csv')
display(question_df.head(5))
display(content_df.head(5))

,ques_id,question
0,1,根据年度报告，2022年中国联通在向数字科技领军企业转变的过程中实现了哪些维度的转型升级？
1,2,告诉我2022年联通产业互联网收入的同比增长速度。
2,3,根据2022年度报告，中国联通的企业定位是什么？
3,4,2022年联通在“大联接”和“大数据”业务上取得了什么成果？
4,5,2022年上半年，联通在精品网络建设上有什么成果？


filename  \
0  AF01.pdf   
1  AF02.pdf   
2  AF04.pdf   
3  AF05.pdf   
4  AF07.pdf   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [66]:
import re

def split_text_into_chunks(text, chunk_size=200):
    sentences = re.split('。|！|？|\n', text)  # 按标点分割句子
    chunks = []
    chunk = []
    for sentence in sentences:
        chunk.append(sentence)
        if len("".join(chunk)) > chunk_size:
            chunks.append("".join(chunk))
            chunk = []
    if chunk:
        chunks.append("".join(chunk))
    return chunks

# 将每个文档进行分块处理
content_df['chunks'] = content_df['content'].apply(split_text_into_chunks)

In [67]:
from modelscope import snapshot_download
model_dir = snapshot_download("AI-ModelScope/bge-large-zh-v1.5", revision='master')

2024-09-19 14:51:25,938 - modelscope - WARNING - Using branch: master as version is unstable, use with caution


In [68]:
from FlagEmbedding import FlagModel
# 加载模型
model = FlagModel(model_dir, 
                  query_instruction_for_retrieval="为这个句子生成表示以用于检索相关文章：",
                  use_fp16=True)

# 使用 FlagModel 对文本块进行向量化
def embed_text(text_list, model):
    embeddings = model.encode(text_list)
    return embeddings

# 对每个文本块进行向量化
content_df['chunk_embeddings'] = content_df['chunks'].apply(
    lambda chunks: embed_text(chunks, model)
)

# 打印部分结果
print(content_df.head())

----------using 2*GPUs----------
   filename  \
0  AF01.pdf   
1  AF02.pdf   
2  AF04.pdf   
3  AF05.pdf   
4  AF07.pdf   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [69]:
import numpy as np

# 检索问题答案
def retrieve_answer(query, model, content_embeddings, content_chunks):
    # 对问题进行向量化
    query_embedding = model.encode([query])[0]
    
    # 计算问题与所有文本块之间的相似度（点积）
    similarities = content_embeddings @ query_embedding.T
    
    # 找到相似度最高的文本块
    top_index = np.argmax(similarities)
    return content_chunks[top_index], similarities[top_index]

# 针对每个问题进行检索
answers = []
for i, row in question_df.iterrows():
    query = row['question']  # 使用 question.csv 中的答案作为查询
    content_embeddings = np.vstack(content_df['chunk_embeddings'].values)
    content_chunks = sum(content_df['chunks'].values, [])  # 将所有块拼接为一个大列表
    
    top_answer, top_similarity = retrieve_answer(query, model, content_embeddings, content_chunks)
    
    # 保存检索结果
    answers.append({
        "question_id": row['ques_id'],
        "answer": top_answer,
        "similarity": top_similarity
    })

# 打印部分答案
for answer in answers[:5]:
    print(f"Question ID: {answer['question_id']}, Answer: {answer['answer']}, Similarity: {answer['similarity']}")


Question ID: 1, Answer: 行业的健康发展贡献了自己的力量综上所述，中国联通在技术创新和智能化生态的打造方面取得了显著成效未来，随着数字经济的不断深入发展，中国联通将继续秉持创新驱动的发展理念，不断提升数据管理能力和智能化水平，为行业的数字化转型和智能化升级贡献更多力量四、数字化转型实践，助力数字中国建设中国联通在数字化转型方面的实践成果丰硕通过加快推进数字化转型，中国联通以改革创新为动力，以集约化建设为契机，坚持推进核心技术自主化，向全面智能化方向演进, Similarity: 0.66357421875
Question ID: 2, Answer: 打造云业务能力平台以及产业互联网解决方案和产品，并联合重点行业头部客户积极打造典型5G应用场景，加快孵化5G行业创新应用，为未来增长积蓄动能2020年第一季度，产业互联网业务收入为人民币114.48亿元，比去年同期上升32.2%得益于创新业务快速增长拉动，固网主营业务收入达到人民币298.94亿元，比去年同期上升11.1%整体主营业务收入为人民币683.07亿元，比去年同期上升2.3%2020年第一季度，网间结算成本比去年同期下降15.6%，主要受网间话务量下滑影响, Similarity: 0.6025390625
Question ID: 3, Answer: 姓名 李玉焯 雷晓旭办公地址 北京市西城区金融大街21号 北京市西城区金融大街21号电话 010-66259179 010-66259179电子信箱 dongmi@chinaunicom.cn ir@chinaunicom.cn2 报告期公司主要业务简介2022年是党的二十大召开之年，也是中国联通全面实施新战略规划的起跑之年，中国联通坚持以习近平新时代中国特色社会主义思想为指导，认真学习宣传贯彻党的二十大精神，坚决贯彻落实党中, Similarity: 0.6982421875
Question ID: 4, Answer: 链安一体化能力不断深化、更加完备四是大数据创新应用行动计划快速推进，服务数字政府的“大数据”能力全面提升中国联通在连续15年为中央级电子政务骨干网提供高质量服务，承建20多个省、100多个省市两级政务云，筑牢数字政府坚实底座的基础上，坚持向上生长、加快创新驱动，依托一点集中和技术领先的大数据能力，发挥数据治理和数

In [70]:
sub_df = pd.read_csv('/data/gyx/2024BDCI/基于运营商文本数据的知识库检索/submit_example.csv')
sub_df

ques_id                                                      question  \
0         1                 根据年度报告，2022年中国联通在向数字科技领军企业转变的过程中实现了哪些维度的转型升级？   
1         2                                     告诉我2022年联通产业互联网收入的同比增长速度。   
2         3                                      根据2022年度报告，中国联通的企业定位是什么？   
3         4                                2022年联通在“大联接”和“大数据”业务上取得了什么成果？   
4         5                                     2022年上半年，联通在精品网络建设上有什么成果？   
..      ...                                                           ...   
95       96                               2018年中国联通大幅下调了国际漫游资费，具体体现在哪些方面？   
96       97                        2017年5月，中国联通两度大幅降低国际漫游资费，两次分别以什么为主要特征？   
97       98  2017年，围绕服务“一带一路”沿线国家产业互联网发展，中国联通面向六类重点行业客户，打造了解决方案，分别是哪六类行业？   
98       99                      中国联通以“三强化三坚持”掀起大兴调查研究热潮重点“三强化三坚持”分别是指什么？   
99      100              截至2017年12月26日，北京市2022年冬奥会和冬残奥会官方合作伙伴已达五家，分别是哪几家？   

                                                                                                                                                                                                                                                                                                                           answer  \
0     我们坚定践行网络强国、数字中国、智慧社会战略部署，今天的中国联通，正在从传统运营商加速向数字科技领军企业转变，实现了四个维度的转型升级：一是联接规模和联接结构升维，从过去的连接人为主拓展到连接人机物，大力发展物联网和工业互联网；二是核心功能升维，从以基础连接为主发展到大联接、大计算、大数据、大应用、大安全五大主责主业；三是服务和赋能水平升维，以5G、云计算、大数据、人工智能、区块链为代表的新一代信息技术和实体经济的结合，服务数字政府、数字社会、数字经济的能力不断增强；四是发展理念升维，我们以传统的市场驱动为主转变为市场驱动和创新驱动相结合的发展模式，尤其是加大了科技创新及人才方面的投入力度，创新发展的动能得到了空前的释放。   
1     我们坚定践行网络强国、数字中国、智慧社会战略部署，今天的中国联通，正在从传统运营商加速向数字科技领军企业转变，实现了四个维度的转型升级：一是联接规模和联接结构升维，从过去的连接人为主拓展到连接人机物，大力发展物联网和工业互联网；二是核心功能升维，从以基础连接为主发展到大联接、大计算、大数据、大应用、大安全五大主责主业；三是服务和赋能水平升维，以6G、云计算、大数据、人工智能、区块链为代表的新一代信息技术和实体经济的结合，服务数字政府、数字社会、数字经济的能力不断增强；四是发展理念升维，我们以传统的市场驱动为主转变为市场驱动和创新驱动相结合的发展模式，尤其是加大了科技创新及人才方面的投入力度，创新发展的动能得到了空前的释放。   
2     我们坚定践行网络强国、数字中国、智慧社会战略部署，今天的中国联通，正在从传统运营商加速向数字科技领军企业转变，实现了四个维度的转型升级：一是联接规模和联接结构升维，从过去的连接人为主拓展到连接人机物，大力发展物联网和工业互联网；二是核心功能升维，从以基础连接为主发展到大联接、大计算、大数据、大应用、大安全五大主责主业；三是服务和赋能水平升维，以7G、云计算、大数据、人工智能、区块链为代表的新一代信息技术和实体经济的结合，服务数字政府、数字社会、数字经济的能力不断增强；四是发展理念升维，我们以传统的市场驱动为主转变为市场驱动和创新驱动相结合的发展模式，尤其是加大了科技创新及人才方面的投入力度，创新发展的动能得到了空前的释放。   
3     我们坚定践行网络强国、数字中国、智慧社会战略部署，今天的中国联通，正在从传统运营商加速向数字科技领军企业转变，实现了四个维度的转型升级：一是联接规模和联接结构升维，从过去的连接人为主拓展到连接人机物，大力发展物联网和工业互联网；二是核心功能升维，从以基础连接为主发展到大联接、大计算、大数据、大应用、大安全五大主责主业；三是服务和赋能水平升维，以8G、云计算、大数据、人工智能、区块链为代表的新一代信息技术和实体经济的结合，服务数字政府、数字社会、数字经济的能力不断增强；四是发展理念升维，我们以传统的市场驱动为主转变为市场驱动和创新驱动相结合的发展模式，尤其是加大了科技创新及人才方面的投入力度，创新发展的动能得到了空前的释放。   
4     我们坚定践行网络强国、数字中国、智慧社会战略部署，今天的中国联通，正在从传统运营商加速向数字科技领军企业转变，实现了四个维度的转型升级：一是联接规模和联接结构升维，从过去的连接人为主拓展到连接人机物，大力发展物联网和工业互联网；二是核心功能升维，从以基础连接为主发展到大联接、大计算、大数据、大应用、大安全五大主责主业；三是服务和赋能水平升维，以9G、云计算、大数据、人工智能、区块链为代表的新一代信息技术和实体经济的结合，服务数字政府、数字社会、数字经济的能力不断增强；四是发展理念升维，我们以传统的市场驱动为主转变为市场驱动和创新驱动相结合的发展模式，尤其是加大了科技创新及人才方面的投入力度，创新发展的动能得到了空前的释放。   
..                                                                                                                                                                                                                                                                                                                            ...   
95  我们坚定践行网络强国、数字中国、智慧社会战略部署，今天的中国联通，正在从传统运营商加速向数字科技领军企业转变，实现了四个维度的转型升级：一是联接规模和联接结构升维，从过去的连接人为主拓展到连接人机物，大力发展物联网和工业互联网；二是核心功能升维，从以基础连接为主发展到大联接、大计算、大数据、大应用、大安全五大主责主业；三是服务和赋能水平升维，以100G、云计算、大数据、人工智能、区块链为代表的新一代信息技术和实体经济的结合，服务数字政府、数字社会、数字经济的能力不断增强；四是发展理念升维，我们以传统的市场驱动为主转变为市场驱动和创新驱动相结合的发展模式，尤其是加大了科技创新及人才方面的投入力度，创新发展的动能得到了空前的释放。   
96  我们坚定践行网络强国、数字中国、智慧社会战略部署，今天的中国联通，正在从传统运营商加速向数字科技领军企业转变，实现了四个维度的转型升级：一是联接规模和联接结构升维，从过去的连接人为主拓展到连接人机物，大力发展物联网和工业互联网；二是核心功能升维，从以基础连接为主发展到大联接、大计算、大数据、大应用、大安全五大主责主业；三是服务和赋能水平升维，以101G、云计算、大数据、人工智能、区块链为代表的新一代信息技术和实体经济的结合，服务数字政府、数字社会、数字经济的能力不断增强；四是发展理念升维，我们以传统的市场驱动为主转变为市场驱动和创新驱动相结合的发展模式，尤其是加大了科技创新及人才方面的投入力度，创新发展的动能得到了空前的释放。   
97  我们坚定践行网络强国、数字中国、智慧社会战略部署，今天的中国联通，正在从传统运营商加速向数字科技领军企业转变，实现了四个维度的转型升级：一是联接规模和联接结构升维，从过去的连接人为主拓展到连接人机物，大力发展物联网和工业互联网；二是核心功能升维，从以基础连接为主发展到大联接、大计算、大数据、大应用、

In [71]:
from tqdm import tqdm
for answer in tqdm(answers):
    sub_df.loc[sub_df['ques_id'] == answer['question_id'], 'answer'] = answer['answer']
sub_df

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 2077.83it/s]


ques_id                                                      question  \
0         1                 根据年度报告，2022年中国联通在向数字科技领军企业转变的过程中实现了哪些维度的转型升级？   
1         2                                     告诉我2022年联通产业互联网收入的同比增长速度。   
2         3                                      根据2022年度报告，中国联通的企业定位是什么？   
3         4                                2022年联通在“大联接”和“大数据”业务上取得了什么成果？   
4         5                                     2022年上半年，联通在精品网络建设上有什么成果？   
..      ...                                                           ...   
95       96                               2018年中国联通大幅下调了国际漫游资费，具体体现在哪些方面？   
96       97                        2017年5月，中国联通两度大幅降低国际漫游资费，两次分别以什么为主要特征？   
97       98  2017年，围绕服务“一带一路”沿线国家产业互联网发展，中国联通面向六类重点行业客户，打造了解决方案，分别是哪六类行业？   
98       99                      中国联通以“三强化三坚持”掀起大兴调查研究热潮重点“三强化三坚持”分别是指什么？   
99      100              截至2017年12月26日，北京市2022年冬奥会和冬残奥会官方合作伙伴已达五家，分别是哪几家？   

                                                                                                                                                                                                                                                    answer  \
0                                  行业的健康发展贡献了自己的力量综上所述，中国联通在技术创新和智能化生态的打造方面取得了显著成效未来，随着数字经济的不断深入发展，中国联通将继续秉持创新驱动的发展理念，不断提升数据管理能力和智能化水平，为行业的数字化转型和智能化升级贡献更多力量四、数字化转型实践，助力数字中国建设中国联通在数字化转型方面的实践成果丰硕通过加快推进数字化转型，中国联通以改革创新为动力，以集约化建设为契机，坚持推进核心技术自主化，向全面智能化方向演进   
1              打造云业务能力平台以及产业互联网解决方案和产品，并联合重点行业头部客户积极打造典型5G应用场景，加快孵化5G行业创新应用，为未来增长积蓄动能2020年第一季度，产业互联网业务收入为人民币114.48亿元，比去年同期上升32.2%得益于创新业务快速增长拉动，固网主营业务收入达到人民币298.94亿元，比去年同期上升11.1%整体主营业务收入为人民币683.07亿元，比去年同期上升2.3%2020年第一季度，网间结算成本比去年同期下降15.6%，主要受网间话务量下滑影响   
2                                  姓名 李玉焯 雷晓旭办公地址 北京市西城区金融大街21号 北京市西城区金融大街21号电话 010-66259179 010-66259179电子信箱 dongmi@chinaunicom.cn ir@chinaunicom.cn2 报告期公司主要业务简介2022年是党的二十大召开之年，也是中国联通全面实施新战略规划的起跑之年，中国联通坚持以习近平新时代中国特色社会主义思想为指导，认真学习宣传贯彻党的二十大精神，坚决贯彻落实党中   
3                             链安一体化能力不断深化、更加完备四是大数据创新应用行动计划快速推进，服务数字政府的“大数据”能力全面提升中国联通在连续15年为中央级电子政务骨干网提供高质量服务，承建20多个省、100多个省市两级政务云，筑牢数字政府坚实底座的基础上，坚持向上生长、加快创新驱动，依托一点集中和技术领先的大数据能力，发挥数据治理和数据安全长板优势，助力各级政府乘“数”而上、借“数”转型我们累计承建15个部委大数据平台、12个省级大数据平台，积极服务全国31   
4   适度加大战略投入，基础网络能力大幅提升中国联通始终坚持网络在企业发展中的基础地位，适度加大战略投入，上半年的精品网络建设卓有成效，为公司有根生长筑牢发展底座5G/4G 精品网建设方面，已经实现重点乡镇以上场景室外连续覆盖，5G中频规模和覆盖水平与行业相当5G新通话已在国内125个城市启动试商用，5G用户满意度保持行业首位；千兆宽带精品网建设方面，网络部署进度与行业同步，FTTH端口达到2.3亿个，南方宽带接入住宅增加近2,500万套，有力推动宽带用户规模和收入规模快速增长；算力精品网建设   
..                                                                                                                                                                                                                                                     ...   
95                             用得起、用得好、用得放心 中国联通发力提升国际漫游用户感知发布时间：2017-07-24发布人：新闻发布人“世界很大，我想去看看”随着全球化的发展，国人出国出境旅游出差越来越多手机的国际漫游也成为国人出国出境游的必选项中国联通从5月31日开始大幅下调国际漫游资费，这是今年第二次国际漫游资费大幅降价此次国际漫游资费调整后，语音漫游资费平均降幅达65.92%，最高降幅达90.42%，如叙利亚语音漫游从29.86元/分钟降至2.86元/分钟   
96                          目前，中国联通已大幅下调“一带一路”区域国际通信资费一是降低国际长途直拨资费中国联通于今年5月下调“一带一路”国家和热门、重点方向的国际长途直拨资费，其中25个国家和地区资费下调至0.49元/分钟二是大幅下调国际漫游资费中国联通已于今年5月下调“一带一路”区域国际漫游资费，数据漫游涉及19个国家地区，最大降幅高达97%语音国际漫游资费调整涉及17个国家和地区，最大降幅达到86%三是优化数据流量资费中国联通不断创新资费模式，通过优化数据包天产   
97                                 网、多云互联等8个产品打造以客户需求为导向的综合行业解决方案围绕服务“一带一路”沿线国家产业互联网发展的需求，中国联通面向金融、交通物流、政府媒体、能源、制造、零售以及互联网等重点行业客户，整合产品和服务，打造以客户需求为导向的综合解决方案积极布局“一带一路”沿线未来智能网络中国联通通过积极探索5G技术、加快部署物联网、推进SDN/NFV网络转型，聚焦智慧工厂、智慧城市、智慧家庭、车联网，推动“一带一路”沿线国家数字化经济发展   
98                              中国联通：以“三强化三坚持”掀起大兴调查研究热潮发布时间：2023-06-09发布人：新闻宣传中心为深入开展学习贯彻习近平新时代中国特色社会主义思想主题教育，中国联通党组牢牢把握“学思想、强党性、重实践、建新功”总要求，把调查研究作为推动企业高质量发展的“金钥匙”，务求“深、实、细、准、效”，在深入开展理论学习的基础上，充分发扬“严实精细快”企业作风，将开展主题教育同贯彻落实习近平总书记重要指示批示精神和党中央重大决策部署结合起来，   
99                               中国联通成为北京2022年冬奥会和冬残奥会官方通信服务合作伙伴发布时间：2017-12-26发布人：新闻发布人12月26日晚间，在中国联通大厦里，中国联通的红色“中国结”与北京2022年冬奥会会徽和冬残奥会会徽的组合标志格外醒目，中国联通与北京2022年冬奥会和冬残奥会组织委员会签约仪式在这里举行这场签约，标志着中国联通正式成为北京2022年冬奥会和冬残奥会官方通信服务合作伙伴，跻身于北京冬奥组委市场开发计划最高级别的赞助企业之列   

                              

In [72]:
# 设置pandas显示的最大列数为None，禁用省略号
pd.set_option('display.max_colwidth', None)
# 禁用科学计数法显示
pd.set_option('display.float_format', '{:.20f}'.format)
# 设置numpy输出时不使用省略号，显示完整的数组内容
np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True, precision=20)
model = FlagModel(model_dir, 
                  query_instruction_for_retrieval="为这个句子生成表示以用于检索相关文章：",
                  use_fp16=True)

# 4. 将sub中的answer列中的文本转换为向量
def embed_text(text_list, model):
    embeddings = model.encode(text_list)
    return embeddings

# 对每个答案进行向量化
sub_df['embedding'] = sub_df['answer'].apply(lambda x: embed_text([x], model)[0])
# 将向量转换为逗号分隔的字符串，并去掉方括号
sub_df['embedding'] = sub_df['embedding'].apply(lambda x: ','.join(map(str, x)))

----------using 2*GPUs----------


In [64]:
# 5. 保存新的sub.csv文件
# sub_df.to_csv('./result/baseline.csv', index=False)